# Multi-Agent HR System - LangGraph & LangChain 1.0

## Overview
Modern multi-agent patterns using **LangGraph** and **LangChain 1.0**:

- **Lab 1**: Basic Multi-Agent with StateGraph (No Tools, No Memory)
- **Lab 2**: Tool Calling Pattern (With Tools, No Memory)  
- **Lab 3**: Handoffs Pattern (With Tools and Memory)
- **Lab 4**: Human-in-the-Loop with Interrupts
- **Lab 5**: Subgraphs - Nested Agent Teams

### Architecture
Using latest LangGraph patterns:
- `StateGraph` for workflow orchestration
- `create_react_agent` for tool-calling agents
- `MemorySaver` for persistence
- `interrupt()` for human-in-the-loop
- Subgraphs for complex multi-agent systems

## Setup and Installation

In [ ]:
# Install required packages
!pip install -q -U langchain-openai langgraph langchain-core python-dotenv

In [ ]:
# Import libraries
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Literal
import operator
from datetime import datetime
import json

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage, ToolMessage
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

# Load environment variables
load_dotenv()

# Verify OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY in your environment")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

print("✓ Setup complete with LangGraph + LangChain 1.0!")

## Sample HR Data

In [ ]:
# Sample candidate data
SAMPLE_RESUME = """
PRIYA SHARMA
Senior Software Engineer
priya.sharma@email.com | +91-98765-43210 | Bengaluru, Karnataka

SUMMARY
6+ years in full-stack development, specializing in Python, React, and AWS.
Led teams, built microservices, mentored developers.

EXPERIENCE
Senior Software Engineer | InfoTech Solutions, Bengaluru | 2021-Present
- Led 4-member team building microservices architecture
- Reduced API latency by 45% through optimization
- Implemented CI/CD pipeline (Jenkins, Docker, K8s)
- Tech: Python, FastAPI, React, PostgreSQL, AWS

Software Engineer | Digital Innovations, Pune | 2018-2020
- Built web apps serving 500K+ users
- Integrated payment gateways (Razorpay, PayU)
- Tech: Python, Django, MySQL, AWS

EDUCATION
B.Tech CSE | BITS Pilani | 2016 | CGPA: 8.5/10

SKILLS
Python, JavaScript, FastAPI, React, AWS, Docker, Kubernetes, PostgreSQL

CERTIFICATIONS
AWS Solutions Architect - Associate (2022)
"""

JOB_DESCRIPTION = """
Senior Backend Engineer - TechCorp India, Bengaluru

Requirements:
- 5+ years backend development
- Python/FastAPI expertise
- AWS/cloud experience
- Microservices architecture
- Team leadership skills

Offer: ₹28-35 LPA + equity
"""

# Mock HR Database
HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "priya.sharma@email.com",
            "phone": "+91-98765-43210",
            "position": "Senior Backend Engineer",
            "status": "screening",
            "resume": SAMPLE_RESUME
        },
        "CAN002": {
            "name": "Arjun Mehta",
            "email": "arjun.mehta@email.com",
            "phone": "+91-98123-45678",
            "position": "Senior Backend Engineer",
            "status": "interview_scheduled"
        }
    },
    "employees": {
        "EMP001": {"name": "Rahul Verma", "role": "Engineering Manager", "email": "rahul.verma@techcorp.in"},
        "EMP002": {"name": "Anjali Patel", "role": "HR Manager", "email": "anjali.patel@techcorp.in"},
        "EMP003": {"name": "Vikram Singh", "role": "Tech Lead", "email": "vikram.singh@techcorp.in"}
    },
    "interview_slots": [
        {"date": "2025-10-15", "time": "10:00", "interviewer": "Rahul Verma", "available": True},
        {"date": "2025-10-15", "time": "14:00", "interviewer": "Vikram Singh", "available": True},
        {"date": "2025-10-16", "time": "11:00", "interviewer": "Anjali Patel", "available": True}
    ]
}

print(f"✓ HR Database loaded: {len(HR_DATABASE['candidates'])} candidates, {len(HR_DATABASE['employees'])} employees")

---

# LAB 1: Basic Multi-Agent with StateGraph (No Tools, No Memory)

**Pattern**: Using LangGraph's StateGraph to coordinate multiple agents

**Flow**:
1. Resume Reviewer → analyzes experience
2. Skills Assessor → evaluates technical fit
3. Decision Maker → final recommendation

Each node is an agent with a specific role.

In [ ]:
# Define state using MessagesState (LangGraph pattern)
class ScreeningState(MessagesState):
    candidate_name: str
    resume: str
    job_description: str
    resume_score: int
    skills_score: int
    final_decision: str

# Agent 1: Resume Reviewer
def resume_reviewer_node(state: ScreeningState):
    """
    Reviews resume and assigns a score
    """
    prompt = f"""
You are a Resume Reviewer at TechCorp India.

Candidate: {state['candidate_name']}

Resume:
{state['resume']}

Job Requirements:
{state['job_description']}

Analyze the resume and provide:
1. Score out of 10 for experience match
2. Key strengths (2-3 points)
3. Concerns (if any)

Start your response with "SCORE: X/10" then provide analysis.
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Extract score (simplified parsing)
    score = 7  # In production, parse from response.content
    if "SCORE: " in response.content:
        try:
            score = int(response.content.split("SCORE: ")[1].split("/")[0])
        except:
            score = 7
    
    return {
        "messages": [AIMessage(content=f"[Resume Reviewer]\n{response.content}", name="resume_reviewer")],
        "resume_score": score
    }

# Agent 2: Skills Assessor
def skills_assessor_node(state: ScreeningState):
    """
    Assesses technical skills match
    """
    # Get previous analysis from messages
    previous_analysis = state['messages'][-1].content if state['messages'] else "No previous analysis"
    
    prompt = f"""
You are a Technical Skills Assessor at TechCorp India.

Candidate: {state['candidate_name']}
Resume Score: {state.get('resume_score', 'N/A')}/10

Previous Analysis:
{previous_analysis}

Resume:
{state['resume']}

Job Requirements:
{state['job_description']}

Evaluate technical skills:
1. Python/FastAPI proficiency
2. AWS/Cloud experience
3. Microservices knowledge
4. Overall technical fit score (1-10)

Start with "SCORE: X/10" then provide detailed assessment.
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Extract score
    score = 8
    if "SCORE: " in response.content:
        try:
            score = int(response.content.split("SCORE: ")[1].split("/")[0])
        except:
            score = 8
    
    return {
        "messages": [AIMessage(content=f"[Skills Assessor]\n{response.content}", name="skills_assessor")],
        "skills_score": score
    }

# Agent 3: Decision Maker
def decision_maker_node(state: ScreeningState):
    """
    Makes final hiring decision based on all analyses
    """
    resume_score = state.get('resume_score', 0)
    skills_score = state.get('skills_score', 0)
    avg_score = (resume_score + skills_score) / 2
    
    # Get all previous messages
    all_analyses = "\n\n".join([msg.content for msg in state['messages']])
    
    prompt = f"""
You are the Hiring Manager at TechCorp India.

Candidate: {state['candidate_name']}
Resume Score: {resume_score}/10
Skills Score: {skills_score}/10
Average Score: {avg_score:.1f}/10

All Analyses:
{all_analyses}

Make final decision:
- If avg_score >= 7: STRONG PROCEED to interview
- If avg_score >= 5: PROCEED with caution
- If avg_score < 5: REJECT

Provide clear recommendation with reasoning.
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Determine decision
    if avg_score >= 7:
        decision = "STRONG_PROCEED"
    elif avg_score >= 5:
        decision = "PROCEED"
    else:
        decision = "REJECT"
    
    return {
        "messages": [AIMessage(content=f"[Hiring Manager]\n{response.content}", name="hiring_manager")],
        "final_decision": decision
    }

# Build the StateGraph
workflow = StateGraph(ScreeningState)

# Add nodes
workflow.add_node("resume_reviewer", resume_reviewer_node)
workflow.add_node("skills_assessor", skills_assessor_node)
workflow.add_node("decision_maker", decision_maker_node)

# Add edges (sequential flow)
workflow.add_edge(START, "resume_reviewer")
workflow.add_edge("resume_reviewer", "skills_assessor")
workflow.add_edge("skills_assessor", "decision_maker")
workflow.add_edge("decision_maker", END)

# Compile
lab1_app = workflow.compile()

print("✓ Lab 1 StateGraph compiled")

In [ ]:
# Test Lab 1
print("\n🧪 Lab 1: Basic Multi-Agent with StateGraph\n")
print("="*80)

input_state = {
    "messages": [],
    "candidate_name": "Priya Sharma",
    "resume": SAMPLE_RESUME,
    "job_description": JOB_DESCRIPTION,
    "resume_score": 0,
    "skills_score": 0,
    "final_decision": ""
}

result = lab1_app.invoke(input_state)

print("\nWorkflow Results:")
print("-" * 80)
for msg in result['messages']:
    print(f"\n{msg.content}")
    print("-" * 80)

print(f"\n✓ Final Decision: {result['final_decision']}")
print(f"✓ Resume Score: {result['resume_score']}/10")
print(f"✓ Skills Score: {result['skills_score']}/10")

---

# LAB 2: Tool Calling Pattern (With Tools, No Memory)

**Pattern**: LangChain 1.0 Tool Calling - agent uses tools as needed

Using `create_react_agent` from LangGraph which creates a ReAct-style agent that can call tools.

In [ ]:
# Define HR Tools

@tool
def get_candidate_info(candidate_id: str) -> str:
    """
    Get candidate information from HR database.
    
    Args:
        candidate_id: Candidate ID like CAN001
    
    Returns:
        JSON string with candidate details
    """
    candidate = HR_DATABASE["candidates"].get(candidate_id)
    if candidate:
        return json.dumps(candidate, indent=2)
    return f"Candidate {candidate_id} not found"

@tool
def search_candidates(position: str) -> str:
    """
    Search for candidates by position.
    
    Args:
        position: Job position to search for
    
    Returns:
        List of matching candidates
    """
    results = []
    for cid, cand in HR_DATABASE["candidates"].items():
        if position.lower() in cand["position"].lower():
            results.append({"id": cid, "name": cand["name"], "status": cand["status"]})
    return json.dumps(results, indent=2)

@tool
def check_interview_availability(interviewer_name: str = None) -> str:
    """
    Check available interview slots.
    
    Args:
        interviewer_name: Optional filter by interviewer
    
    Returns:
        Available interview slots
    """
    slots = HR_DATABASE["interview_slots"]
    
    if interviewer_name:
        slots = [s for s in slots if interviewer_name.lower() in s["interviewer"].lower()]
    
    available = [s for s in slots if s["available"]]
    return json.dumps(available, indent=2)

@tool
def update_candidate_status(candidate_id: str, new_status: str) -> str:
    """
    Update candidate status in ATS.
    
    Args:
        candidate_id: Candidate ID
        new_status: New status (screening, interview_scheduled, offer, rejected)
    
    Returns:
        Confirmation message
    """
    if candidate_id in HR_DATABASE["candidates"]:
        old_status = HR_DATABASE["candidates"][candidate_id]["status"]
        HR_DATABASE["candidates"][candidate_id]["status"] = new_status
        return f"✓ Updated {HR_DATABASE['candidates'][candidate_id]['name']}: {old_status} → {new_status}"
    return f"❌ Candidate {candidate_id} not found"

@tool
def send_email_notification(to_email: str, subject: str, message: str) -> str:
    """
    Send email notification.
    
    Args:
        to_email: Recipient email
        subject: Email subject
        message: Email body
    
    Returns:
        Confirmation
    """
    return f"✓ Email sent to {to_email}\nSubject: {subject}\n[SIMULATED]"

# Collect all tools
hr_tools = [
    get_candidate_info,
    search_candidates,
    check_interview_availability,
    update_candidate_status,
    send_email_notification
]

print("✓ HR Tools defined:")
for t in hr_tools:
    print(f"  - {t.name}")

In [ ]:
# Create ReAct agent with tools (LangGraph pattern)

system_prompt = """
You are a Recruitment Coordinator at TechCorp India.

Your role:
- Search for candidates
- Check candidate information
- Schedule interviews
- Update candidate status
- Send notifications

Use the available tools to complete tasks efficiently.
Always provide clear, professional responses.
"""

# Create agent using create_react_agent
lab2_agent = create_react_agent(
    llm,
    tools=hr_tools,
    state_modifier=system_prompt
)

print("✓ Lab 2 ReAct Agent created with tools")

In [ ]:
# Test Lab 2
print("\n🧪 Lab 2: Tool Calling Pattern\n")
print("="*80)

# Task: Schedule interview for a candidate
task = """
Please help me schedule an interview for candidate CAN001 (Priya Sharma).

Steps needed:
1. Get the candidate's information
2. Check available interview slots with Rahul Verma
3. Update candidate status to 'interview_scheduled'
4. Send email confirmation to the candidate
"""

result = lab2_agent.invoke(
    {"messages": [HumanMessage(content=task)]}
)

print("\nAgent Execution:")
print("-" * 80)
for msg in result['messages']:
    if isinstance(msg, AIMessage) and msg.content:
        print(f"\n🤖 Agent: {msg.content}")
    elif isinstance(msg, ToolMessage):
        print(f"\n🔧 Tool ({msg.name}): {msg.content[:200]}..." if len(msg.content) > 200 else f"\n🔧 Tool ({msg.name}): {msg.content}")

---

# LAB 3: Handoffs Pattern (With Tools and Memory)

**Pattern**: LangChain 1.0 Handoffs - agents pass control to each other

**Features**:
- Multiple specialized agents
- Agents can hand off to each other
- Persistent memory with checkpointing
- Multi-turn conversations

In [ ]:
# Define handoff tool to pass control between agents

@tool
def transfer_to_screening_agent() -> str:
    """Transfer the conversation to the Screening Agent who handles resume reviews and initial assessments."""
    return "Transferring to Screening Agent..."

@tool
def transfer_to_interview_agent() -> str:
    """Transfer to Interview Coordinator who handles interview scheduling and logistics."""
    return "Transferring to Interview Coordinator..."

@tool
def transfer_to_offer_agent() -> str:
    """Transfer to Offer Manager who handles offer preparation and negotiation."""
    return "Transferring to Offer Manager..."

# Agent 1: Screening Agent
screening_agent = create_react_agent(
    llm,
    tools=[get_candidate_info, search_candidates, update_candidate_status, transfer_to_interview_agent],
    state_modifier="""
You are the Screening Agent at TechCorp India.

Your responsibilities:
- Review resumes and assess candidates
- Search for candidates in the database
- Update candidate status after screening

When a candidate passes screening and needs interview scheduling, 
use transfer_to_interview_agent to hand off.
"""
)

# Agent 2: Interview Coordinator
interview_agent = create_react_agent(
    llm,
    tools=[check_interview_availability, update_candidate_status, send_email_notification, transfer_to_offer_agent],
    state_modifier="""
You are the Interview Coordinator at TechCorp India.

Your responsibilities:
- Check interview slot availability
- Schedule interviews
- Send interview confirmations
- Update candidate status

When interview is successful and it's time for offer, 
use transfer_to_offer_agent to hand off.
"""
)

# Agent 3: Offer Manager
offer_agent = create_react_agent(
    llm,
    tools=[get_candidate_info, send_email_notification, update_candidate_status],
    state_modifier="""
You are the Offer Manager at TechCorp India.

Your responsibilities:
- Prepare job offers
- Send offer letters
- Handle offer negotiations
- Update final candidate status
"""
)

print("✓ Three specialized agents created with handoff capabilities")

In [ ]:
# Build multi-agent system with handoffs using StateGraph

class MultiAgentState(MessagesState):
    current_agent: str

def route_after_agent(state: MultiAgentState) -> Literal["screening", "interview", "offer", "__end__"]:
    """
    Route based on the last tool call
    """
    messages = state['messages']
    last_message = messages[-1]
    
    # Check if last message was a tool call for handoff
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        tool_call = last_message.tool_calls[-1]
        if 'transfer_to_interview' in tool_call['name']:
            return "interview"
        elif 'transfer_to_offer' in tool_call['name']:
            return "offer"
        elif 'transfer_to_screening' in tool_call['name']:
            return "screening"
    
    # Check tool messages for handoff indicators
    if isinstance(last_message, ToolMessage):
        if "Interview Coordinator" in last_message.content:
            return "interview"
        elif "Offer Manager" in last_message.content:
            return "offer"
    
    # Default: end conversation
    return "__end__"

# Build workflow
handoff_workflow = StateGraph(MultiAgentState)

# Add agent nodes
handoff_workflow.add_node("screening", screening_agent)
handoff_workflow.add_node("interview", interview_agent)
handoff_workflow.add_node("offer", offer_agent)

# Set entry point
handoff_workflow.add_edge(START, "screening")

# Add conditional edges for handoffs
handoff_workflow.add_conditional_edges(
    "screening",
    route_after_agent,
    {"screening": "screening", "interview": "interview", "offer": "offer", "__end__": END}
)

handoff_workflow.add_conditional_edges(
    "interview",
    route_after_agent,
    {"screening": "screening", "interview": "interview", "offer": "offer", "__end__": END}
)

handoff_workflow.add_conditional_edges(
    "offer",
    route_after_agent,
    {"screening": "screening", "interview": "interview", "offer": "offer", "__end__": END}
)

# Compile with memory
memory = MemorySaver()
lab3_app = handoff_workflow.compile(checkpointer=memory)

print("✓ Lab 3 Multi-Agent Handoff System compiled with memory")

In [ ]:
# Test Lab 3: Multi-turn conversation with handoffs
print("\n🧪 Lab 3: Handoffs Pattern with Memory\n")
print("="*80)

config = {"configurable": {"thread_id": "hiring_session_001"}}

# Turn 1: Start with screening
print("\n--- Turn 1: Initial Screening Request ---")
response1 = lab3_app.invoke(
    {
        "messages": [HumanMessage(content="Please screen candidate CAN001 for the Senior Backend Engineer position.")],
        "current_agent": "screening"
    },
    config
)

print("\nLast AI Response:")
for msg in reversed(response1['messages']):
    if isinstance(msg, AIMessage) and msg.content:
        print(msg.content)
        break

# Turn 2: Continue conversation (agent remembers context)
print("\n\n--- Turn 2: Request Interview Scheduling (Tests Memory) ---")
response2 = lab3_app.invoke(
    {
        "messages": [HumanMessage(content="Great! Now schedule an interview for this candidate with Rahul Verma.")]
    },
    config
)

print("\nLast AI Response:")
for msg in reversed(response2['messages']):
    if isinstance(msg, AIMessage) and msg.content:
        print(msg.content)
        break

print("\n\n--- Conversation History ---")
print(f"Total messages in thread: {len(response2['messages'])}")
print("✓ Agent maintains context across turns using checkpointing!")

---

# LAB 4: Human-in-the-Loop with Interrupts

**Pattern**: LangGraph `interrupt()` for human approval

**Use Case**: Offer approval workflow
- Agent prepares offer
- Graph interrupts for human approval
- Human provides feedback
- Agent continues based on approval

In [ ]:
# Define state for HITL workflow
class OfferApprovalState(MessagesState):
    candidate_id: str
    offer_details: dict
    approval_status: str
    human_feedback: str

# Node 1: Prepare Offer
def prepare_offer_node(state: OfferApprovalState):
    """
    Prepare job offer for candidate
    """
    candidate = HR_DATABASE["candidates"].get(state['candidate_id'], {})
    
    prompt = f"""
You are an HR Compensation Specialist at TechCorp India.

Prepare a job offer for:
- Candidate: {candidate.get('name', 'Unknown')}
- Position: {candidate.get('position', 'Unknown')}

Create offer package with:
1. Base salary: ₹30 LPA
2. Performance bonus: Up to 20%
3. Stock options: 1000 shares (4-year vesting)
4. Benefits: Health insurance, learning budget ₹50k/year
5. Start date: Within 30 days

Format professionally for management review.
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    offer = {
        "candidate": candidate.get('name'),
        "position": candidate.get('position'),
        "base_salary": "₹30 LPA",
        "bonus": "20%",
        "equity": "1000 shares",
        "details": response.content
    }
    
    return {
        "messages": [AIMessage(content=f"Offer prepared:\n\n{response.content}", name="offer_preparer")],
        "offer_details": offer,
        "approval_status": "pending"
    }

# Node 2: Request Human Approval (using interrupt)
def request_approval_node(state: OfferApprovalState):
    """
    Interrupt workflow for human approval
    """
    offer = state['offer_details']
    
    # This will pause the graph execution
    approval_request = {
        "type": "offer_approval",
        "candidate": offer.get('candidate'),
        "offer_details": offer,
        "question": "Do you approve this offer? (approve/reject/modify)"
    }
    
    # Use interrupt to pause for human input
    human_response = interrupt(approval_request)
    
    return {
        "approval_status": human_response.get("decision", "pending"),
        "human_feedback": human_response.get("feedback", "")
    }

# Node 3: Process Approval
def process_approval_node(state: OfferApprovalState):
    """
    Take action based on approval decision
    """
    status = state['approval_status']
    feedback = state.get('human_feedback', '')
    candidate_id = state['candidate_id']
    
    if status == "approve":
        # Update candidate status
        update_candidate_status.invoke({"candidate_id": candidate_id, "new_status": "offer_sent"})
        
        message = f"""
✓ Offer APPROVED!

Next steps:
1. Generate formal offer letter
2. Send to candidate
3. Set follow-up reminder for 3 days

Feedback: {feedback}
"""
    elif status == "reject":
        message = f"""
❌ Offer REJECTED

Reason: {feedback}
No further action taken.
"""
    else:
        message = f"""
⚠️ Modifications requested: {feedback}
Sending back to compensation team for revision.
"""
    
    return {
        "messages": [AIMessage(content=message, name="approval_processor")]
    }

# Build HITL workflow
hitl_workflow = StateGraph(OfferApprovalState)

hitl_workflow.add_node("prepare_offer", prepare_offer_node)
hitl_workflow.add_node("request_approval", request_approval_node)
hitl_workflow.add_node("process_approval", process_approval_node)

hitl_workflow.add_edge(START, "prepare_offer")
hitl_workflow.add_edge("prepare_offer", "request_approval")
hitl_workflow.add_edge("request_approval", "process_approval")
hitl_workflow.add_edge("process_approval", END)

# Compile with checkpointer (required for interrupts)
hitl_memory = MemorySaver()
lab4_app = hitl_workflow.compile(
    checkpointer=hitl_memory,
    interrupt_before=["request_approval"]  # Interrupt before this node
)

print("✓ Lab 4 HITL workflow compiled with interrupt capability")

In [ ]:
# Test Lab 4: Human-in-the-Loop
print("\n🧪 Lab 4: Human-in-the-Loop with Interrupts\n")
print("="*80)

config = {"configurable": {"thread_id": "offer_approval_001"}}

# Step 1: Start workflow (will run until interrupt)
print("\nStep 1: Preparing offer...")
result1 = lab4_app.invoke(
    {
        "messages": [],
        "candidate_id": "CAN001",
        "offer_details": {},
        "approval_status": "",
        "human_feedback": ""
    },
    config
)

print("\nOffer Prepared:")
print("-" * 80)
for msg in result1['messages']:
    if isinstance(msg, AIMessage):
        print(msg.content)

# Check if interrupted
state_snapshot = lab4_app.get_state(config)
print(f"\nWorkflow Status: {'INTERRUPTED (awaiting approval)' if state_snapshot.next else 'COMPLETED'}")

if state_snapshot.next:
    print("\n" + "="*80)
    print("HUMAN APPROVAL REQUIRED")
    print("="*80)
    print("\nSimulating human approval...")
    
    # Step 2: Resume with human feedback
    # In production, this would come from a UI/API
    human_decision = {
        "decision": "approve",
        "feedback": "Offer looks good. Approved by management."
    }
    
    # Update state with human response and continue
    result2 = lab4_app.invoke(
        Command(
            update={
                "approval_status": human_decision["decision"],
                "human_feedback": human_decision["feedback"]
            }
        ),
        config
    )
    
    print("\nFinal Result After Approval:")
    print("-" * 80)
    for msg in result2['messages']:
        if isinstance(msg, AIMessage) and msg.name == "approval_processor":
            print(msg.content)

print("\n✓ HITL workflow demonstrates proper interrupt and resume pattern!")

---

# LAB 5: Subgraphs - Nested Agent Teams

**Pattern**: LangGraph Subgraphs for complex multi-agent systems

**Architecture**:
- **Parent Graph**: Overall hiring pipeline orchestration
- **Subgraph 1**: Screening Team (resume review → skills assessment → decision)
- **Subgraph 2**: Interview Team (panel selection → scheduling → confirmation)

Subgraphs share state with parent graph through common keys.

In [ ]:
# ============================================================================
# SUBGRAPH 1: Screening Team
# ============================================================================

class ScreeningTeamState(MessagesState):
    candidate_id: str
    resume: str
    screening_decision: str  # Shared with parent
    screening_notes: str

def quick_screen_node(state: ScreeningTeamState):
    """Quick resume screening"""
    prompt = f"Screen resume quickly (PASS/FAIL only):\n{state['resume'][:400]}..."
    response = llm.invoke([HumanMessage(content=prompt)])
    
    decision = "PASS" if "pass" in response.content.lower() else "FAIL"
    
    return {
        "messages": [AIMessage(content=f"Quick Screen: {response.content}", name="quick_screener")],
        "screening_notes": response.content
    }

def detailed_assessment_node(state: ScreeningTeamState):
    """Detailed skills assessment"""
    if "FAIL" in state.get('screening_notes', ''):
        return {"screening_decision": "REJECT"}
    
    prompt = f"Assess technical skills (score 1-10):\n{state['resume'][:400]}..."
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "messages": [AIMessage(content=f"Skills Assessment: {response.content}", name="skills_assessor")]
    }

def screening_decision_node(state: ScreeningTeamState):
    """Final screening decision"""
    # Analyze all messages in screening team
    notes = "\n".join([m.content for m in state['messages'] if isinstance(m, AIMessage)])
    
    # Simple decision logic
    if "FAIL" in notes or "fail" in notes.lower():
        decision = "REJECT"
    elif any(word in notes.lower() for word in ["strong", "excellent", "pass"]):
        decision = "PROCEED_TO_INTERVIEW"
    else:
        decision = "PROCEED_WITH_CAUTION"
    
    return {
        "screening_decision": decision,
        "messages": [AIMessage(content=f"Screening Decision: {decision}", name="decision_maker")]
    }

# Build screening subgraph
screening_team = StateGraph(ScreeningTeamState)
screening_team.add_node("quick_screen", quick_screen_node)
screening_team.add_node("detailed_assessment", detailed_assessment_node)
screening_team.add_node("make_decision", screening_decision_node)

screening_team.add_edge(START, "quick_screen")
screening_team.add_edge("quick_screen", "detailed_assessment")
screening_team.add_edge("detailed_assessment", "make_decision")
screening_team.add_edge("make_decision", END)

screening_subgraph = screening_team.compile()

print("✓ Screening Team Subgraph compiled")

In [ ]:
# ============================================================================
# SUBGRAPH 2: Interview Team  
# ============================================================================

class InterviewTeamState(MessagesState):
    candidate_id: str
    candidate_name: str
    interview_scheduled: bool  # Shared with parent
    interview_details: str  # Shared with parent

def select_panel_node(state: InterviewTeamState):
    """Select interview panel"""
    panel = "Rahul Verma (Engineering Manager), Vikram Singh (Tech Lead)"
    return {
        "messages": [AIMessage(content=f"Panel selected: {panel}", name="panel_selector")]
    }

def schedule_interview_node(state: InterviewTeamState):
    """Schedule interview"""
    # Use the tool to check availability
    slots = check_interview_availability.invoke({"interviewer_name": "Rahul"})
    
    details = f"""
Interview Scheduled for {state.get('candidate_name', 'Candidate')}
Date: 2025-10-15
Time: 10:00 AM IST  
Duration: 90 minutes
Panel: Rahul Verma, Vikram Singh
Meeting Link: https://meet.google.com/abc-defg-hij
"""
    
    return {
        "interview_scheduled": True,
        "interview_details": details,
        "messages": [AIMessage(content=f"Interview scheduled:\n{details}", name="scheduler")]
    }

def send_confirmations_node(state: InterviewTeamState):
    """Send interview confirmations"""
    candidate = HR_DATABASE["candidates"].get(state['candidate_id'], {})
    
    # Send emails (simulated)
    send_email_notification.invoke({
        "to_email": candidate.get('email', ''),
        "subject": "Interview Invitation - TechCorp India",
        "message": state['interview_details']
    })
    
    return {
        "messages": [AIMessage(content="Confirmations sent to candidate and panel", name="notifier")]
    }

# Build interview subgraph
interview_team = StateGraph(InterviewTeamState)
interview_team.add_node("select_panel", select_panel_node)
interview_team.add_node("schedule", schedule_interview_node)
interview_team.add_node("send_confirmations", send_confirmations_node)

interview_team.add_edge(START, "select_panel")
interview_team.add_edge("select_panel", "schedule")
interview_team.add_edge("schedule", "send_confirmations")
interview_team.add_edge("send_confirmations", END)

interview_subgraph = interview_team.compile()

print("✓ Interview Team Subgraph compiled")

In [ ]:
# ============================================================================
# PARENT GRAPH: Complete Hiring Pipeline
# ============================================================================

class HiringPipelineState(MessagesState):
    candidate_id: str
    candidate_name: str
    resume: str
    screening_decision: str  # Shared with screening subgraph
    screening_notes: str
    interview_scheduled: bool  # Shared with interview subgraph
    interview_details: str  # Shared with interview subgraph
    pipeline_status: str

def initialize_pipeline_node(state: HiringPipelineState):
    """Initialize the hiring pipeline"""
    candidate = HR_DATABASE["candidates"].get(state['candidate_id'], {})
    
    return {
        "candidate_name": candidate.get('name', 'Unknown'),
        "resume": candidate.get('resume', SAMPLE_RESUME),
        "messages": [AIMessage(content=f"Starting pipeline for {candidate.get('name')}", name="coordinator")],
        "pipeline_status": "initialized"
    }

def route_after_screening(state: HiringPipelineState) -> Literal["interview_team", "reject"]:
    """Route based on screening decision"""
    decision = state.get('screening_decision', '')
    
    if 'PROCEED' in decision:
        return "interview_team"
    return "reject"

def reject_candidate_node(state: HiringPipelineState):
    """Handle candidate rejection"""
    return {
        "pipeline_status": "rejected",
        "messages": [AIMessage(
            content=f"Candidate {state['candidate_name']} rejected at screening.\nReason: {state['screening_decision']}",
            name="coordinator"
        )]
    }

def finalize_pipeline_node(state: HiringPipelineState):
    """Finalize the pipeline"""
    if state.get('interview_scheduled'):
        status = "interview_scheduled"
        message = f"""
✓ Pipeline Complete for {state['candidate_name']}

Screening: {state['screening_decision']}
Interview: {state['interview_details']}

Status: Ready for interview
"""
    else:
        status = state.get('pipeline_status', 'completed')
        message = f"Pipeline completed with status: {status}"
    
    return {
        "pipeline_status": status,
        "messages": [AIMessage(content=message, name="coordinator")]
    }

# Build parent graph
parent_graph = StateGraph(HiringPipelineState)

# Add nodes (including subgraphs as nodes!)
parent_graph.add_node("initialize", initialize_pipeline_node)
parent_graph.add_node("screening_team", screening_subgraph)  # SUBGRAPH!
parent_graph.add_node("interview_team", interview_subgraph)  # SUBGRAPH!
parent_graph.add_node("reject", reject_candidate_node)
parent_graph.add_node("finalize", finalize_pipeline_node)

# Add edges
parent_graph.add_edge(START, "initialize")
parent_graph.add_edge("initialize", "screening_team")

# Conditional routing after screening
parent_graph.add_conditional_edges(
    "screening_team",
    route_after_screening,
    {"interview_team": "interview_team", "reject": "reject"}
)

parent_graph.add_edge("interview_team", "finalize")
parent_graph.add_edge("reject", "finalize")
parent_graph.add_edge("finalize", END)

# Compile
lab5_app = parent_graph.compile()

print("✓ Lab 5 Complete Pipeline with Subgraphs compiled!")
print("  - Parent Graph: Hiring Pipeline Orchestrator")
print("  - Subgraph 1: Screening Team (3 agents)")
print("  - Subgraph 2: Interview Team (3 agents)")

In [ ]:
# Test Lab 5: Complete Pipeline with Subgraphs
print("\n🧪 Lab 5: Subgraphs - Complete Hiring Pipeline\n")
print("="*80)

# Run complete pipeline
result = lab5_app.invoke(
    {
        "messages": [],
        "candidate_id": "CAN001",
        "candidate_name": "",
        "resume": "",
        "screening_decision": "",
        "screening_notes": "",
        "interview_scheduled": False,
        "interview_details": "",
        "pipeline_status": ""
    },
    {"configurable": {"thread_id": "pipeline_001"}}
)

print("\n📊 Pipeline Execution Flow:")
print("-" * 80)
for i, msg in enumerate(result['messages'], 1):
    if isinstance(msg, AIMessage):
        agent_name = msg.name if hasattr(msg, 'name') else 'Unknown'
        print(f"\n{i}. [{agent_name}]")
        print(f"   {msg.content[:200]}..." if len(msg.content) > 200 else f"   {msg.content}")

print("\n\n📈 Final Pipeline State:")
print("-" * 80)
print(f"Candidate: {result['candidate_name']}")
print(f"Screening Decision: {result['screening_decision']}")
print(f"Interview Scheduled: {result['interview_scheduled']}")
print(f"Pipeline Status: {result['pipeline_status']}")

print("\n✓ Demonstrates parent graph coordinating multiple specialized subgraph teams!")

---

## 🎯 Summary: LangGraph + LangChain 1.0 Patterns

### Lab 1: Basic Multi-Agent with StateGraph ✅
- Pure LangGraph `StateGraph` orchestration
- Sequential agent coordination
- State sharing between agents
- No external dependencies

### Lab 2: Tool Calling Pattern ✅
- `create_react_agent` for tool-enabled agents
- ReAct-style reasoning with tools
- Dynamic tool selection by LLM
- Real HR operations simulation

### Lab 3: Handoffs Pattern with Memory ✅
- Multiple specialized agents
- Agent-to-agent handoffs
- `MemorySaver` for persistent state
- Multi-turn conversation context
- Thread-based checkpointing

### Lab 4: Human-in-the-Loop ✅
- LangGraph `interrupt()` mechanism
- Workflow pause for human input
- `interrupt_before` configuration
- Resume execution with `Command`
- Production-ready approval patterns

### Lab 5: Subgraphs (Advanced) ✅
- Nested graph architecture
- Subgraphs as nodes in parent graph
- State sharing through common keys
- Complex multi-team coordination
- Scalable agent system design

## 🚀 Key Technologies

- **LangGraph**: StateGraph, create_react_agent, MemorySaver, interrupt()
- **LangChain 1.0**: Tool calling, message types, state management
- **OpenAI**: GPT-4o for agent reasoning
- **Patterns**: Tool calling, handoffs, HITL, subgraphs

## 📚 References

- [LangChain Multi-Agent](https://docs.langchain.com/oss/python/langchain/multi-agent)
- [LangGraph Subgraphs](https://docs.langchain.com/oss/python/langgraph/use-subgraphs)
- [Human-in-the-Loop](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)

All labs use **modern LangGraph and LangChain 1.0 patterns** as documented! 🎉